# Project Title: Retail Insights Engine


#### submitted by : Gururaj H C

## Overview: 
You have been contracted by a mid-size retail business that has been collecting sales and inventory data over the last couple of years. 

They want to utilize this data to gain insights into their business.

Your task is to build a Retail Insights Engine that will take in their historical data, process it, analyze it, and provide actionable insights.


In [1]:
#importing the necessary libraries
import pandas as pd
import numpy as np
from itertools import combinations
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

In [2]:
#loading the dataset
#pd.set_option('display.max_rows',None)
df = pd.read_excel('C:\MLCourse\Coding_practice@Emplay\Week-02\Data.xlsx',parse_dates=True)
df.head()

,BillNo,Itemname,Quantity,Date,Price,CustomerID
0,558638,ZINC METAL HEART DECORATION,12,2011-07-01 08:16:00,1.25,16317.0
1,558638,HANGING JAM JAR TLIGHT HOLDER,24,2011-07-01 08:16:00,0.85,16317.0
2,558638,LANTERN CREAM GAZEBO,3,2011-07-01 08:16:00,4.95,16317.0
3,558638,ZINC T-LIGHT HOLDER STAR LARGE,12,2011-07-01 08:16:00,0.95,16317.0
4,558638,FRENCH TOILET SIGN BLUE METAL,12,2011-07-01 08:16:00,1.25,16317.0


In [3]:
#Checking for abnormal data
df.describe()

,Quantity,Price,CustomerID
count,110813.000000,110813.000000,82340.000000
mean,10.142700,3.507632,15552.756789
std,44.150431,63.053221,1589.067844
min,-3667.000000,-11062.060000,12747.000000
25%,1.000000,1.250000,14191.000000
50%,4.000000,2.080000,15529.000000
75%,12.000000,4.130000,16910.000000
max,3186.000000,11062.060000,18283.000000


#### Negative quantity means returned items and negative price means adjusted values. They need to be dropped 

In [4]:
#Checking for null values in each columns
df.isnull().sum()

BillNo            0
Itemname        336
Quantity          0
Date              0
Price             0
CustomerID    28473
dtype: int64

In [5]:
#Checking for total null values in the df
df.isnull().sum().sum()

28809

More than 25% of the CustomerID is null. instead of dropping or imputing the values, to maintain sanctity of the data mentioning the missing values as 99999. So, that they can be easily identfied

### Preprocessing

In [6]:
# Fill missing values in 'Itemname' and 'CustomerID'
df['Itemname'] = df['Itemname'].fillna('UNKNOWN ITEM')
df['CustomerID'] = df['CustomerID'].fillna(99999)

# Convert 'CustomerID' to integers
df['CustomerID'] = df['CustomerID'].astype(int)

# Drop rows with negative 'Quantity' or 'Price'
df = df[(df['Quantity'] >= 0) & (df['Price'] >= 0)]

# Remove rows where 'Itemname' contains one or more question marks
df = df[~df['Itemname'].str.contains(r'\?+')]

# Extract year, month, day, hour, and day of the week from 'Date'
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['Hour'] = df['Date'].dt.hour
df['day_of_week'] = df['Date'].dt.day_name()

# Remove 'A' from 'BillNo' and convert to integers
# Remove 'A' from 'BillNo' and convert to integers
for i, row in df.iterrows():
    if 'BillNo' in df.columns and len(str(row['BillNo'])) > 6:
        df.at[i, 'BillNo'] = str(row['BillNo']).replace('A', '')

df['BillNo'] = df['BillNo'].astype(int)

In [7]:
df

,BillNo,Itemname,Quantity,Date,Price,CustomerID,Year,Month,Day,Hour,day_of_week
0,558638,ZINC METAL HEART DECORATION,12,2011-07-01 08:16:00,1.25,16317,2011,7,1,8,Friday
1,558638,HANGING JAM JAR TLIGHT HOLDER,24,2011-07-01 08:16:00,0.85,16317,2011,7,1,8,Friday
2,558638,LANTERN CREAM GAZEBO,3,2011-07-01 08:16:00,4.95,16317,2011,7,1,8,Friday
3,558638,ZINC T-LIGHT HOLDER STAR LARGE,12,2011-07-01 08:16:00,0.95,16317,2011,7,1,8,Friday
4,558638,FRENCH TOILET SIGN BLUE METAL,12,2011-07-01 08:16:00,1.25,16317,2011,7,1,8,Friday
...,...,...,...,...,...,...,...,...,...,...,...
110808,569202,PLASMATRONIC LAMP,1,2011-09-30 17:22:00,8.29,99999,2011,9,30,17,Friday
110809,569202,SET OF 2 ROUND TINS CAMEMBERT,1,2011-09-30 17:22:00,5.79,99999,2011,9,30,17,Friday
110810,569202,MINI JIGSAW DOLLY GIRL,2,2011-09-30 17:22:00,0.83,99999,2011,9,30,17,Friday
110811,569202,MINI JIGSAW CIRCUS PARADE,2,2011-09-30 17:22:00,0.83,99999,2011,9,30,17,Friday


## Business Analysis

### Which product are the top performers?

In [8]:
print('Displaying the top 5 best selling products')
df.groupby('Itemname')['Quantity'].sum().sort_values(ascending=False).head(5)

Displaying the top 5 best selling products


Itemname
JUMBO BAG RED RETROSPOT               11857
ASSORTED COLOUR BIRD ORNAMENT         10942
POPCORN HOLDER                        10604
WORLD WAR 2 GLIDERS ASSTD DESIGNS      7714
WHITE HANGING HEART T-LIGHT HOLDER     7304
Name: Quantity, dtype: int64

### Which products are lagging?


In [9]:
print('Displaying the top 5 worst selling products')
df.groupby('Itemname')['Quantity'].sum().sort_values().head(5)

Displaying the top 5 worst selling products


Itemname
FELT FARM ANIMAL RABBIT                1
GOLD/AMBER DROP EARRINGS W LEAF        1
SET 10 CARDS ROBIN WATERPUMP  17096    1
LADYBIRD + BEE RAFFIA FOOD COVER       1
SET 10 CARDS SNOWY SNOWDROPS  17100    1
Name: Quantity, dtype: int64

### The business has a hunch that certain products sell better at specific times of the year or maybe even days of the week. Using the data, validate this hunch.


In [10]:
df.groupby(['Itemname','Month'])['Quantity'].sum().sort_values(ascending=False)

Itemname                           Month
ASSORTED COLOUR BIRD ORNAMENT      8        6497
POPCORN HOLDER                     8        5135
GIRLS ALPHABET IRON ON PATCHES     7        5019
JUMBO BAG RED RETROSPOT            8        4748
                                   9        4061
                                            ... 
SET/3 CHRISTMAS DECOUPAGE CANDLES  7           1
COSY SLIPPER SHOES LARGE GREEN     7           1
4 PINK FLOCK CHRISTMAS BALLS       9           1
4 PURPLE FLOCK DINNER CANDLES      8           1
NUMBER TILE COTTAGE GARDEN 7       9           1
Name: Quantity, Length: 7864, dtype: int64

In [11]:
df_item = df[df['Itemname'] == 'ASSORTED COLOUR BIRD ORNAMENT']
df_item.groupby('Month')['Quantity'].sum().sort_values(ascending=False)

Month
8    6497
9    2712
7    1733
Name: Quantity, dtype: int64

In [12]:
df_item = df[df['Itemname'] == 'ASSORTED COLOUR BIRD ORNAMENT']
df_item.groupby('day_of_week')['Quantity'].sum().sort_values(ascending=False)

day_of_week
Thursday     6258
Monday       1156
Tuesday       912
Wednesday     910
Friday        882
Sunday        824
Name: Quantity, dtype: int64

#### The hunch is true. For instance : ASSORTED COLOUR BIRD ORNAMENT,  sold the most in August and on thursdays

### Are there products that were once best-sellers but have seen a decrease in sales recently?

In [13]:
df.groupby(['Itemname','Month'])['Quantity'].sum().sort_values(ascending=False)

Itemname                           Month
ASSORTED COLOUR BIRD ORNAMENT      8        6497
POPCORN HOLDER                     8        5135
GIRLS ALPHABET IRON ON PATCHES     7        5019
JUMBO BAG RED RETROSPOT            8        4748
                                   9        4061
                                            ... 
SET/3 CHRISTMAS DECOUPAGE CANDLES  7           1
COSY SLIPPER SHOES LARGE GREEN     7           1
4 PINK FLOCK CHRISTMAS BALLS       9           1
4 PURPLE FLOCK DINNER CANDLES      8           1
NUMBER TILE COTTAGE GARDEN 7       9           1
Name: Quantity, Length: 7864, dtype: int64

In [14]:
df_3 = df[df['Itemname'] == 'GIRLS ALPHABET IRON ON PATCHES']
df_3.groupby('Month')['Quantity'].sum()

Month
7    5019
8     179
9    1550
Name: Quantity, dtype: int64

####  The product GIRLS ALPHABET IRON ON PATCHES was sold 5k in July and saw decline in August and Sepetember

### Identify the most common products that are bought together. Can you find any product pairs or combinations?

In [15]:
df10 = df[['BillNo', 'Itemname', 'CustomerID']]
grouped = df10.groupby(['BillNo', 'CustomerID'])['Itemname'].apply(list).reset_index()
product_pairs = []
for items in grouped['Itemname']:
    item_combinations = list(combinations(items, 2))
    product_pairs.extend(item_combinations)
product_pair_counts = Counter(product_pairs)
most_common_pairs = product_pair_counts.most_common(20)  
result_df = pd.DataFrame(most_common_pairs, columns=['Product Pair', 'Frequency'])
result_df

,Product Pair,Frequency
0,"(JUMBO BAG APPLES, JUMBO BAG RED RETROSPOT)",147
1,"(JUMBO BAG PINK POLKADOT, JUMBO BAG RED RETROS...",142
2,"(LUNCH BAG RED RETROSPOT, LUNCH BAG BLACK SKU...",128
3,"(JUMBO STORAGE BAG SUKI, JUMBO BAG RED RETROSPOT)",127
4,"(LUNCH BAG SUKI DESIGN, LUNCH BAG APPLE DESIGN)",123
5,"(JUMBO SHOPPER VINTAGE RED PAISLEY, JUMBO BAG ...",120
6,"(SPOTTY BUNTING, PARTY BUNTING)",119
7,"(LUNCH BAG SPACEBOY DESIGN, LUNCH BAG SUKI DES...",117
8,"(LUNCH BAG RED RETROSPOT, LUNCH BAG SUKI DESIGN)",117
9,"(JUMBO STORAGE BAG SUKI, JUMBO BAG APPLES)",117


### Which are the top 10 most frequent product baskets?
 

In [16]:
result_df.head(10)

,Product Pair,Frequency
0,"(JUMBO BAG APPLES, JUMBO BAG RED RETROSPOT)",147
1,"(JUMBO BAG PINK POLKADOT, JUMBO BAG RED RETROS...",142
2,"(LUNCH BAG RED RETROSPOT, LUNCH BAG BLACK SKU...",128
3,"(JUMBO STORAGE BAG SUKI, JUMBO BAG RED RETROSPOT)",127
4,"(LUNCH BAG SUKI DESIGN, LUNCH BAG APPLE DESIGN)",123
5,"(JUMBO SHOPPER VINTAGE RED PAISLEY, JUMBO BAG ...",120
6,"(SPOTTY BUNTING, PARTY BUNTING)",119
7,"(LUNCH BAG SPACEBOY DESIGN, LUNCH BAG SUKI DES...",117
8,"(LUNCH BAG RED RETROSPOT, LUNCH BAG SUKI DESIGN)",117
9,"(JUMBO STORAGE BAG SUKI, JUMBO BAG APPLES)",117


### Identify different baskets of goods sold, which are the top 10 baskets? 

In [17]:
df11 = df[['BillNo', 'Itemname', 'CustomerID']]
baskets = df11.groupby(['BillNo', 'CustomerID'])['Itemname'].unique().reset_index()
sorted_baskets = baskets.sort_values(by='Itemname', key=lambda x: x.str.len(), ascending=False)
top_10_baskets = sorted_baskets.head(10)
print(top_10_baskets)

      BillNo  CustomerID                                           Itemname
188   559055       99999  [SWEETIES  STICKERS, HOME BUILDING BLOCK WORD,...
187   559052       99999  [3D SHEET OF DOG STICKERS, PACK 6 HEART/ICE-CR...
893   560772       99999  [RED HEART LUGGAGE TAG, DOORMAT RED RETROSPOT,...
779   560434       99999  [GARDEN PATH JOURNAL, DENIM PATCH PURSE PINK B...
972   560926       99999  [PAINTED METAL PEARS ASSORTED, NO JUNK MAIL ME...
499   559816       99999  [COLOURING PENCILS BROWN TUBE, ASSTD DESIGN RA...
3728  567656       14096  [PINK DIAMANTE PEN IN GIFT BOX, CRYSTAL SEA HO...
3560  567183       14769  [WOOD BLACK BOARD ANT WHITE FINISH, STOOL HOME...
662   560225       99999  [JUMBO  BAG BAROQUE BLACK WHITE, WHITE HANGING...
456   559693       99999  [LUNCH BAG RED RETROSPOT, GOLD MINI TAPE MEASU...


### How does basket size vary with the day of the week or time of the month?

In [18]:
print('Basket size during the day of the week')
df.groupby(['day_of_week'])['Quantity'].sum().sort_values(ascending=False)

Basket size during the day of the week


day_of_week
Thursday     285026
Tuesday      224759
Wednesday    207829
Friday       163049
Monday       156731
Sunday       121340
Name: Quantity, dtype: int64

In [19]:
print('Basket size during days of the month')
df.groupby(['Day'])['Quantity'].sum().sort_values(ascending=False)

Basket size during days of the month


Day
20    58963
11    56919
4     56669
28    53865
22    50334
5     49396
24    44254
21    43594
25    43232
15    41671
19    40631
1     39479
12    39047
26    38293
18    38290
13    37621
7     36847
29    36599
8     36409
17    34619
14    32298
27    31363
2     28776
23    28750
30    28203
6     27919
16    26295
9     21734
31    20846
3     18717
10    17101
Name: Quantity, dtype: int64

### Identify different baskets of goods sold, which are the top 10 baskets?


In [21]:
basket_counts = df.groupby('BillNo')['Itemname'].nunique().reset_index(name='UniqueItems')
top_10_baskets = basket_counts[basket_counts['UniqueItems'] > 1].sort_values(by='UniqueItems', ascending=False).head(10)
print(top_10_baskets)

      BillNo  UniqueItems
188   559055          487
187   559052          463
893   560772          449
779   560434          443
972   560926          438
499   559816          433
3728  567656          419
3560  567183          386
662   560225          379
456   559693          358


In [22]:
df12 = df[['BillNo', 'Itemname']]
baskets = df12.groupby('BillNo')['Itemname'].apply(set)
basket_counts = baskets.value_counts().reset_index()
basket_counts.columns = ['Basket', 'Count']
top_10_baskets = basket_counts.head(10)
def get_item_names(basket):
    return ', '.join(sorted(list(basket)))
top_10_baskets['ItemNames'] = top_10_baskets['Basket'].apply(get_item_names)
result_df = top_10_baskets[['ItemNames', 'Count']].reset_index(drop=True)
result_df

,ItemNames,Count
0,UNKNOWN ITEM,135
1,Manual,25
2,POSTAGE,21
3,found,8
4,WHITE HANGING HEART T-LIGHT HOLDER,6
5,PLEASE ONE PERSON METAL SIGN,6
6,DOORMAT KEEP CALM AND COME IN,5
7,PANTRY MAGNETIC SHOPPING LIST,5
8,CHILLI LIGHTS,4
9,POPCORN HOLDER,4


### Are there any loyal customers? Identify customers who make frequent purchases.

In [23]:
print('Displaying the Top 5 customers')
df4 = df[df['CustomerID']!=99999.0]
df4.groupby('CustomerID')['Quantity'].sum().sort_values(ascending=False).head(5)

Displaying the Top 5 customers


CustomerID
17450    35468
12931    18114
18102    17333
17511    16310
13694    13646
Name: Quantity, dtype: int64

### Can you segment customers based on their total spend?

In [24]:
df['Total'] = df['Quantity']*df['Price']
df_cust = df.groupby('CustomerID')['Total'].sum().reset_index()
df_filtered = df_cust[df_cust['CustomerID'] != 99999.0]
df_filtered = df_filtered.sort_values('Total').reset_index()
df_filtered['Spending_category'] = ''
for i in range(len(df_filtered)):
    if df_filtered['Total'][i] < 100:
        df_filtered['Spending_category'][i] = 'Low'
    elif (df_filtered['Total'][i] >= 100) & (df_filtered['Total'][i] < 500):
        df_filtered['Spending_category'][i] = 'Medium'
    elif (df_filtered['Total'][i] >= 500) & (df_filtered['Total'][i] < 1000):
        df_filtered['Spending_category'][i] = 'High'
    else:
        df_filtered['Spending_category'][i] = 'Very high'
print('Printing the category and number of customers in each category based on thier spending : ')
pd.value_counts(df_filtered['Spending_category'])

Printing the category and number of customers in each category based on thier spending : 


Medium       986
High         446
Very high    405
Low          102
Name: Spending_category, dtype: int64

### How often do customers return to make another purchase?

In [25]:
grouped = df.groupby('CustomerID')
df2 = pd.DataFrame({'CustomerID': grouped.groups.keys(), 'Data': grouped['Date'].apply(list)})
df2 = df2.reset_index(drop=True)
def remove_duplicates(dates_list):
    return list(set(dates_list))
df2['Data'] = df2['Data'].apply(remove_duplicates)
df2 = df2.explode('Data', ignore_index=True)
df2['Data'] = pd.to_datetime(df2['Data'])
df2.reset_index(drop=True, inplace=True)
df2 = df2[df2['CustomerID']!= 99999.0]
df2 = df2.sort_values(['CustomerID','Data'])
df2 = df2.reset_index(drop=True)
df2['Time_Diff'] = df2.groupby('CustomerID')['Data'].diff()
df2['Time_Diff'].fillna(pd.Timedelta(seconds=0), inplace=True)
df2.reset_index(drop=True, inplace=True)
print(f'On an average the customers return every : {df2["Time_Diff"].mean()} days for a fresh purchase')


On an average the customers return every : 9 days 09:01:30.380139643 days for a fresh purchase
